# 2020 Data
### LMI Capstone Team
### Summer Chambers | Steve Morris | Kaleb Shikur

## Setup

In [13]:
import pandas as pd
import numpy as np
import json
import math
from statistics import mode
from sklearn.metrics import ndcg_score, f1_score, average_precision_score
from matplotlib import pyplot as plt
import seaborn as sns
import elasticsearch
from elasticsearch import Elasticsearch, RequestsHttpConnection, ElasticsearchException
import regex as re
from bs4 import BeautifulSoup
import requests #gets urls
import time
import csv
import os
import sys
import string
from random import sample

In [2]:
# 173.79.72.92, port 9200
# host = '173.79.72.92'
endpoint = 'https://search-lmi-capstone-2-525zkk33t4z5iy6ozqd63ctgmq.us-east-1.es.amazonaws.com'

In [3]:
#es = Elasticsearch(endpoint, timeout = 45)
es = Elasticsearch(endpoint, timeout=600, max_retries=2, retry_on_timeout=True)



print(es.info())

# es = Elasticsearch(
#     hosts=[{'host': host, 'port': 443}],
#     http_auth=auth,
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )
# print(es.info())

{'name': 'eb7b3348d3c34ce7dc22f263930c51ad', 'cluster_name': '846033058400:lmi-capstone-2', 'cluster_uuid': 'QzTffcNgStmet053IRSP2w', 'version': {'number': '7.9.1', 'build_flavor': 'oss', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2020-11-03T09:54:32.349659Z', 'build_snapshot': False, 'lucene_version': '8.6.2', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


## True Sections

In [4]:
metadata_url = "https://mikeanders.org/data/CMS/CMS-2020-0088-1604/CMS-2020-0088-1604%20Comment%20MetaData.xlsx"
data = pd.read_excel(metadata_url)
data["comment_id"] = [i for i, comment in enumerate(data.Primary)]
data = data.replace(["EUC Policy for 2021", "Comment Solicitation: EUC Policy for 2022 and beyond"], \
             "Comment Solicitation: EUC Policy for 2021 and beyond") #combining two sections that can't be split in rule

In [5]:
data.head()

,Primary,Tracking Number,Organization,Topic,Supportive?,Comment text,Unique Flag,HeavyHitter,Modified,Form Letter?,FormLetter Y/N,Sheet Name,comment_id
0,CMS-2020-0088-9495,1k4-9ivz-162j,MarsdenAdvisors,Applying the APM APP to SSP ACOs,Mixed,We are part of a MIPS APM that consistently sc...,1,NaN,2020-09-14 14:32:16,NaN,NaN,Liz Tracker,0
1,CMS-2020-0088-9494,1k4-9ivz-50bx,MarsdenAdvisors,Applying the APM APP to SSP ACOs,Mixed,We are part of a MIPS APM that consistently sc...,1,NaN,2020-09-14 14:32:16,NaN,NaN,Liz Tracker,1
2,CMS-2020-0088-8582,kev-e48h-ghk4,Kari Kerstetter,Applying the APM APP to SSP ACOs,Opposed,Thank you for the opportunity to provide publi...,1,NaN,2020-10-09 11:23:30,NaN,NaN,Dylan Tracker,2
3,CMS-2020-0088-8582,kev-e48h-ghk4,Kari Kerstetter,Revising the SSP Quality Performance Standard,Opposed,"...In addition, the move to a 4th decile ACO q...",0,NaN,2020-10-09 11:23:30,NaN,NaN,Dylan Tracker,3
4,CMS-2020-0088-31162,1k4-9jcr-57bp,"PBACO Holding, LLC (PBACO)",Applying the APM APP to SSP ACOs,Opposed,We respectfully decline from supporting CMS pr...,1,NaN,2020-10-08 18:18:49,NaN,NaN,Eric Tracker,4


### Topic Count

In [6]:
data.Topic.value_counts()

Applying the APM APP to SSP ACOs                                                             245
Revising the SSP Quality Performance Standard                                                 71
Updating Primary Care Services Definition for Assignment                                      48
Comment Solicitation: EUC Policy for 2020                                                     41
Proposed Changes to CAHPS for 2020                                                            33
Comment Solicitation: EUC Policy for 2021 and beyond                                          29
Other: Unsolicited                                                                            25
Quality Redesign: Use of ACO Quality Performance in Determining Shared Savings and Losses     16
Revising Policy for Determining the Amount of Repayment Mechanism                             15
Applicability of Policies to Track 1+ Model ACOs                                               4
Quality Redesign: Compliance w

## Rule-Splitting

In [7]:
rule_url = "https://mikeanders.org/data/CMS/CMS-2020-0088-1604/Rule/CMS-2020-0088-1604.txt"

In [8]:
startdict = {"Applying the APM APP to SSP ACOs": ["b. Applying the Alternative Payment Model (APM) Performance Pathway (APP) to Shared Savings Program ACOs", \
                                                  "c. Shared Savings Program Quality Performance Standard"], \
           "Revising the SSP Quality Performance Standard": ["c. Shared Savings Program Quality Performance Standard", \
                                                             "d. Use of ACO Quality Performance in Determining Shared Savings and Shared Losses"], \
           "Comment Solicitation: EUC Policy for 2020": ["2. Comment Solicitation on Modifications to the Extreme and Uncontrollable Circumstances Policy for Performance Year 2020", \
                                                         "J. Proposal To Remove Selected National Coverage Determinations"], \
           "Updating Primary Care Services Definition for Assignment": ["2. Revisions to the Definition of Primary Care Services used in Shared Savings Program Beneficiary Assignment", \
                                                                        "3. Reducing the Amount of Repayment Mechanisms for Eligible ACOs"], \
           "Quality Redesign: Use of ACO Quality Performance in Determining Shared Savings and Losses": ["d. Use of ACO Quality Performance in Determining Shared Savings and Shared Losses", \
                                                                                                         "e. Compliance With the Quality Performance Standard"], \
           "Quality Redesign: Compliance with the Quality Performance Standard": ["e. Compliance With the Quality Performance Standard", \
                                                                                  "f. Updating the Process Used To Validate ACO Quality Data Reporting"], \
           "Proposed Changes to CAHPS for 2020": ["1. Proposed Changes to the CAHPS for ACOs Reporting Requirements for Performance Year 2020", \
                                                  "2. Comment Solicitation on Modifications to the Extreme and Uncontrollable Circumstances Policy for Performance Year 2020"], \
           "Applicability of Policies to Track 1+ Model ACOs": ["4. Applicability of Policies to Track 1+ Model ACOs", \
                                                                "I. Modifications to Quality Reporting Requirements and Comment Solicitation on Modifications to the Extreme and Uncontrollable Circumstances Policy for Performance Year 2020"], \
           "Revising Policy for Determining the Amount of Repayment Mechanism": ["3. Reducing the Amount of Repayment Mechanisms for Eligible ACOs", \
                                                                                 "4. Applicability of Policies to Track 1+ Model ACOs"], \
           "Quality Redesign: Proposed Technical Changes to Quality Regulation Text":  ["h. Proposed Technical Changes To Incorporate References to Revised Quality Performance Standard", \
                                                                                        "2. Revisions to the Definition of Primary Care Services Used in Shared Savings Program Beneficiary Assignment"], \
           "Comment Solicitation: EUC Policy for 2021 and beyond": ["g. Changes to the Extreme and Uncontrollable Circumstances Policy for Performance Year 2021", \
                                                                    "h. Proposed Technical Changes To Incorporate References to Revised Quality Performance Standard"], \
           "Updating the Process Used to validate ACO Quality Data Reporting": ["f. Updating the Process Used To Validate ACO Quality Data Reporting", \
                                                                                "g. Changes to the Extreme and Uncontrollable Circumstances Policy for Performance Year 2021"] }

## Headers Split

In [9]:
def splitRule_headers(rule_url, startdict):

    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings
    for key in startdict.keys():
        startdict[key] = [val.lower() for val in startdict[key]]
    
    rulechunks = []
    
    for key, value in startdict.items():    
       splitlist = alltxt.split(value[0]) #split on start of desired section
       split_further = splitlist[1].split(value[1]) #split again on start of undesired section
       rulechunks.append({"section": key, "text": (value[0]+split_further[0])}) #choose only first half to upload to dict

    return rulechunks

In [10]:
split1 = splitRule_headers(rule_url, startdict)
headers_id_list = [doc["section"] for doc in split1]
lengths = {chunk["section"]: len(chunk["text"]) for chunk in split1}
print("sections:", len(lengths))
lengths

sections: 12


{'Applying the APM APP to SSP ACOs': 23299,
 'Revising the SSP Quality Performance Standard': 7592,
 'Comment Solicitation: EUC Policy for 2020': 10933,
 'Updating Primary Care Services Definition for Assignment': 45692,
 'Quality Redesign: Use of ACO Quality Performance in Determining Shared Savings and Losses': 9614,
 'Quality Redesign: Compliance with the Quality Performance Standard': 18527,
 'Proposed Changes to CAHPS for 2020': 8890,
 'Applicability of Policies to Track 1+ Model ACOs': 9995,
 'Revising Policy for Determining the Amount of Repayment Mechanism': 22281,
 'Quality Redesign: Proposed Technical Changes to Quality Regulation Text': 3041,
 'Comment Solicitation: EUC Policy for 2021 and beyond': 11322,
 'Updating the Process Used to validate ACO Quality Data Reporting': 2638}

## Hybrid Split

In [11]:
def splitRule_line_hybrid(rule_url):
    new_rule_chunks = []
    chunks = splitRule_headers(rule_url, startdict)
    for doc in chunks:
        paragraphs = doc["text"].split('\r\n')
        #add new lines while under 6000 characters
        for i in range(len(paragraphs) - 1):
            while i < (len(paragraphs)-1) and len(paragraphs[i]) < 6000:
                paragraphs[i] += paragraphs[i+1]
                del(paragraphs[i+1])
        for i in range(len(paragraphs)):
            new_rule_chunks.append({"section": doc["section"]+"_"+str(i), "text": paragraphs[i]})
    return new_rule_chunks

In [12]:
split2 = splitRule_line_hybrid(rule_url)
hybrid_id_list = [doc["section"] for doc in split2]
lengths = {chunk["section"]: len(chunk["text"]) for chunk in split2}
print("sections:", len(lengths))
lengths

sections: 32


{'Applying the APM APP to SSP ACOs_0': 6267,
 'Applying the APM APP to SSP ACOs_1': 8189,
 'Applying the APM APP to SSP ACOs_2': 6735,
 'Applying the APM APP to SSP ACOs_3': 2046,
 'Revising the SSP Quality Performance Standard_0': 6873,
 'Revising the SSP Quality Performance Standard_1': 703,
 'Comment Solicitation: EUC Policy for 2020_0': 6352,
 'Comment Solicitation: EUC Policy for 2020_1': 4557,
 'Updating Primary Care Services Definition for Assignment_0': 6489,
 'Updating Primary Care Services Definition for Assignment_1': 6307,
 'Updating Primary Care Services Definition for Assignment_2': 6107,
 'Updating Primary Care Services Definition for Assignment_3': 8129,
 'Updating Primary Care Services Definition for Assignment_4': 7356,
 'Updating Primary Care Services Definition for Assignment_5': 6711,
 'Updating Primary Care Services Definition for Assignment_6': 4391,
 'Quality Redesign: Use of ACO Quality Performance in Determining Shared Savings and Losses_0': 6169,
 'Quality Re

## Upload Splits to ES

In [152]:
##es.indices.delete(index='2020hybrid_1shard', ignore=[400, 404])

In [153]:
def rulesplit_toES(rulechunks, es_index):
    for chunk in rulechunks:
            res = es.index(index=es_index, id=chunk["section"], body=chunk, doc_type='_doc')
            es.indices.refresh(index=es_index)
    print("Last id uploaded:", chunk["section"])

In [154]:
rulesplit_1shard = splitRule_headers(rule_url, startdict)
upload_1shard = rulesplit_toES(rulesplit_1shard, "2020headers_1shard")

Last id uploaded: Updating the Process Used to validate ACO Quality Data Reporting


In [155]:
rulesplit_1shard = splitRule_line_hybrid(rule_url)
upload_1shard = rulesplit_toES(rulesplit_1shard, "2020hybrid_1shard")

Last id uploaded: Updating the Process Used to validate ACO Quality Data Reporting_0


## Term Vectors 

#### Get top common words

In [13]:
termVectorBody = {
  "fields" : ["text"],
  "term_statistics" : True,
  "field_statistics" : True,
  "offsets" : False,
  "payloads" : False,
  "positions" : False,
    "filter": {
    "max_num_terms": 30,
    "min_term_freq": 1,
    "min_doc_freq": 11 ## 90% of documents
  }
}

In [14]:
termvecs = dict()
for section in headers_id_list:
    result = es.termvectors(index = "2020headers_1shard", id = section, body= termVectorBody)
    termvecs[section] = list(result['term_vectors']['text']['terms'].keys())

In [16]:
common_words = [value for key, value in termvecs.items()]
common_words = list(set([word for list1 in common_words for word in list1]))
common_words

['current',
 'believ',
 'continu',
 'determin',
 'begin',
 'period',
 'therefor',
 'chang',
 'applic',
 'address',
 'data',
 'elig',
 'ar',
 '2021']

#### Examine top words

In [17]:
termVectorBody = {
  "fields" : ["text"],
  "term_statistics" : True,
  "field_statistics" : True,
  "offsets" : False,
  "payloads" : False,
  "positions" : False,
    "filter": {
    "max_num_terms": 10,
    "min_term_freq": 1,
    "min_doc_freq": 1,
    "max_doc_freq": 10
  }
}

In [18]:
termvecs = dict()
for section in headers_id_list:
    result = es.termvectors(index = "2020headers_1shard", id = section, body= termVectorBody)
    termvecs[section] = list(result['term_vectors']['text']['terms'].keys())

In [19]:
for key, value in termvecs.items():
    print(key, "\n", value, "\n\n")

Applying the APM APP to SSP ACOs 
 ['align', 'apm', 'app', 'categori', 'measur', 'mip', 'qualiti', 'report', 'score', 'set'] 


Revising the SSP Quality Performance Standard 
 ['40th', 'categori', 'exclud', 'mip', 'percentil', 'qp', 'qualiti', 'score', 'specifi', 'standard'] 


Comment Solicitation: EUC Policy for 2020 
 ['2020', 'complet', 'extrem', 'impact', 'measur', 'phe', 'qualiti', 'report', 'score', 'uncontrol'] 


Updating Primary Care Services Definition for Assignment 
 ['assign', 'care', 'code', 'cpt', 'hcpc', 'manag', 'primari', 'profession', 'servic', 'snf'] 


Quality Redesign: Use of ACO Quality Performance in Determining Shared Savings and Losses 
 ['d', 'enhanc', 'level', 'loss', 'model', 'percent', 'qualiti', 'rate', 'side', 'track'] 


Quality Redesign: Compliance with the Quality Performance Standard 
 ['425.316', 'agreement', 'consecut', 'enter', 'fail', 'meet', 'qualiti', 'renew', 'standard', 'termin'] 


Proposed Changes to CAHPS for 2020 
 ['2020', 'administr', 

## EXCEL COMMENTS

In [20]:
excel2020 = {num:text for num, text in zip(data["comment_id"], data["Comment text"])}

### Write to .json

In [21]:
dump = json.dumps(excel2020)
f = open('excel2020.json','w')
f.write(dump)
f.close()

## Remove duplicate excel comments

In [22]:
list_excel2020 = list(excel2020.values())

In [23]:
from DeDuper import *
duplicates = getDupes(list_excel2020)
print(len(duplicates))

96


In [304]:
#duplicates

### Examine Duplicates

In [24]:
dupdict = {}
for i, dup in enumerate(duplicates):
    dupdict[i] = [list_excel2020[idx][0:120] for idx in dup]

In [195]:
#dupdict

## Create Truth Dictionary of Unique Comments

In [25]:
dup_topics = {}
for dup_list in duplicates: #iterate through the duplicate lists
    topic_list = []
    for dup in dup_list: #iterate through the comments
        topic_list.append(data.loc[data["comment_id"] == dup, 'Topic'].item()) #find topic for that comment
    dup_topics[int(dup)] = list(set(topic_list)) #choose last comment in list, map it to only unique topics in that duplicates list

In [26]:
truth_dict = dict(sorted(dup_topics.items()))

### Write truth_dict

In [27]:
dump = json.dumps(truth_dict)
f = open('truth_dict2020.json','w')
f.write(dump)
f.close()

### Read truth_dict (avoid running duplicates)

In [40]:
with open('truth_dict2020.json') as f:
    truth_dict = json.load(f, parse_int=int)
    truth_dict = {int(key):value for key, value in truth_dict.items()}

## Get only unique comments text, make lowercase, replace \t etc.

In [42]:
unique_excel2020 = {key:value.lower().replace('\t', " ").replace('\n', " ").replace('\r', " ").replace('\s', " ")\
                    for key, value in excel2020.items() if key in list(truth_dict.keys())}

### Write unique comments

In [44]:
dump = json.dumps(unique_excel2020)
f = open('unique_excel2020.json','w')
f.write(dump)
f.close()

### Read unique comments (avoid running duplicates)

In [46]:
with open('unique_excel2020.json') as f:
    unique_excel2020 = json.load(f)
    unique_excel2020 = {int(key):value for key, value in unique_excel2020.items()}

## Create Unique DataFrame with topics as columns, ones and zeroes

In [63]:
topics = list(data.Topic.dropna().unique())

In [64]:
topic_df = data.copy()
for topic in topics:
    topic_df[topic] = np.zeros(len(data.Topic))
topic_df = topic_df[topic_df.comment_id.isin(list(unique_excel2020.keys()))]

In [65]:
for key, value in truth_dict.items():
    for topic in topics:
        if topic in value:
            topic_df[topic][key] = 1
topic_df[topics] = topic_df[topics].astype(int)

<ipython-input-65-aaa36271f3c5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topic_df[topic][key] = 1


In [66]:
topic_df

,Primary,Tracking Number,Organization,Topic,Supportive?,Comment text,Unique Flag,HeavyHitter,Modified,Form Letter?,...,Comment Solicitation: EUC Policy for 2021 and beyond,Updating Primary Care Services Definition for Assignment,Other: Unsolicited,Quality Redesign: Use of ACO Quality Performance in Determining Shared Savings and Losses,Quality Redesign: Compliance with the Quality Performance Standard,Proposed Changes to CAHPS for 2020,Applicability of Policies to Track 1+ Model ACOs,Revising Policy for Determining the Amount of Repayment Mechanism,Quality Redesign: Proposed Technical Changes to Quality Regulation Text,Updating the Process Used to validate ACO Quality Data Reporting
0,CMS-2020-0088-9495,1k4-9ivz-162j,MarsdenAdvisors,Applying the APM APP to SSP ACOs,Mixed,We are part of a MIPS APM that consistently sc...,1,NaN,2020-09-14 14:32:16,NaN,...,0,0,0,0,0,0,0,0,0,0
10,CMS-2020-0088-31162,1k4-9jcr-57bp,"PBACO Holding, LLC (PBACO)",Applying the APM APP to SSP ACOs,Opposed,PBACO opposes the removal of Pay-for-Reporting...,0,NaN,2020-10-08 18:18:49,NaN,...,0,1,0,0,0,0,0,0,0,0
12,CMS-2020-0088-31162,1k4-9jcr-57bp,"PBACO Holding, LLC (PBACO)",Comment Solicitation: EUC Policy for 2020,Supportive,Awarding the Higher of the 2019 or 2020 ACO Qu...,0,NaN,2020-10-08 18:18:49,NaN,...,0,0,0,0,0,0,0,0,0,0
14,CMS-2020-0088-31162,1k4-9jcr-57bp,"PBACO Holding, LLC (PBACO)",Comment Solicitation: EUC Policy for 2021 and ...,Opposed,Adjustments to the MSSP Extreme and Uncontroll...,0,NaN,2020-10-07 13:13:33,NaN,...,1,1,0,0,0,0,0,0,0,0
15,CMS-2020-0088-31162,1k4-9jcr-57bp,"PBACO Holding, LLC (PBACO)",Updating Primary Care Services Definition for ...,Supportive,PBACO supports updating the definition of prim...,0,NaN,2020-10-07 13:14:23,NaN,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,CMS-2020-0088-16118,1k4-9j7v-cjoa,Community Health Provider Alliance (CHPA),Applying the APM APP to SSP ACOs,Opposed,"Additionally, moving to the alternative report...",0,NaN,2020-09-30 09:33:12,NaN,...,0,0,0,0,0,0,0,0,0,0
437,CMS-2020-0088-16118,1k4-9j7v-cjoa,Community Health Provider Alliance (CHPA),Updating Primary Care Services Definition for ...,Other - Unsolicited,CHPA also advocates for expansion of telehealt...,0,NaN,2020-09-30 09:33:12,NaN,...,0,1,0,0,0,0,0,0,0,0
460,CMS-2020-0088-16087,1k4-9j7v-fauk,Community Health Provider Alliance (CHPA),Quality Redesign: Use of ACO Quality Performan...,Opposed,"Additionally, CHPA opposes the overall approac...",0,1.0,2020-09-30 13:59:01,NaN,...,0,0,0,1,0,0,0,0,0,0
466,CMS-2020-0088-16087,1k4-9j7v-fauk,Community Health Provider Alliance (CHPA),Revising Policy for Determining the Amount of ...,Supportive,Under the Pathways to Success regulations when...,0,1.0,2020-09-30 13:59:01,NaN,...,0,0,0,0,0,0,0,1,0,0


## SCRAPE COMMENTS

In [100]:
comment_url = "https://mikeanders.org/data/CMS/CMS-2020-0088-1604/Comments/"

In [107]:
response = requests.get(comment_url)
soup = BeautifulSoup(response.text, 'html.parser')
a_tags = soup.findAll("a")
links = [tag["href"] for tag in a_tags]
txt_links = [link for link in links if '.txt' in link]
comments = {}
txt_sampled = sample(txt_links, 400)
for suffix in txt_sampled:
    comments[suffix[14:19]] = requests.get(comment_url+suffix).text.lower()
    print(suffix[14:19], end=" ")

27263 14757 4081. 12407 9318. 7951_ 6346. 22950 25087 10310 19016 11720 5169. 11847 26170 16844 19308 5615_ 16732 29378 10132 21984 9460. 31050 27775 24212 5148. 23098 5446. 30719 20607 18248 3580. 6017. 17690 3873. 16401 11983 7781. 29013 30443 25489 13666 7303. 28679 26773 14237 26913 27652 2383. 29581 4959. 7359. 6914. 14712 7314. 6766. 17430 2725. 30465 22797 22507 2862. 11882 19051 1740. 26872 21773 13936 7426_ 12483 17048 8911. 24176 3100. 25565 21589 4851. 28195 4280. 2722. 25510 20831 22954 3805. 22975 27428 7039. 21402 28609 12890 26777 4139. 25190 23669 16279 20372 23417 18884 15849 26058 7196. 31315 15342 17861 20653 12522 23079 6932. 20846 24710 31121 16031 18438 2271. 29538 28145 2571. 12597 8130. 5170. 21016 21611 19436 10114 23130 28470 28359 7163. 24545 8714. 24722 12659 6621. 25738 29803 25459 23234 12218 21052 23291 7433. 25182 25817 12374 15718 23309 16215 8790. 5153. 9578_ 22221 8782. 22024 26763 11121 29343 14207 15870 3601. 4388. 13142 10516 13236 29831 16123 4046

### Write to .json

In [120]:
comment_json = json.dumps(comments)
f = open('comments2020.json','w')
f.write(comment_json)
f.close()

### Read .json (avoid scraping comments)

In [121]:
with open('comments2020.json') as f:
    comments2020 = json.load(f)

## Remove duplicate comments, comments less than 30 characters long

In [124]:
reduced_comments2020 = {key: val for key, val in comments2020.items() if len(val) >= 30}
short_ones = {key: val for key, val in comments2020.items() if len(val) < 30}
list_reduced_comments2020 = list(reduced_comments2020.values())
len(reduced_comments2020)
short_ones #these seem okay to remove

{'26913': '\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n',
 '22797': 'please see attached letter',
 '8911.': 'see attached file(s)',
 '24176': 'see attached file(s)',
 '28195': 'see attached file(s)',
 '22954': 'see attached file(s)',
 '22975': 'please see attached file.',
 '31315': 'see attached',
 '2571.': 'see attached file(s)',
 '13590': 'see attached file(s)',
 '24553': 'see attached file(s)',
 '19450': 'see attached file(s)',
 '20209': 'see attached file(s)',
 '13417': 'please see attached file.',
 '14666': 'see attached file(s)',
 '30264': 'my letter is attached.',
 '10753': 'see attached file(s)',
 '14093': 'see attached',
 '14129': 'see attached file(s)'}

In [126]:
from DeDuper import *
duplicates = getDupes(list_reduced_comments2020)
print(len(duplicates))
#duplicates

23


### Examine Duplicates

In [128]:
dupdict = {}
for i, dup in enumerate(duplicates):
    dupdict[i] = [list_reduced_comments2020[idx][0:120] for idx in dup]

In [130]:
#dupdict

### Remove Them

In [131]:
tb_deleted = []
for num, duplist in enumerate(duplicates):
    for idx, comment in enumerate(list_reduced_comments2020):
        if idx in duplist[:-1]:
            tb_deleted.append(comment)

In [132]:
len(tb_deleted)

306

In [133]:
unique_comments2020 = {key:value for key, value in reduced_comments2020.items() if value not in tb_deleted}

In [134]:
len(unique_comments2020)

68

### Write to .json

In [135]:
dump = json.dumps(unique_comments2020)
f = open('unique_comments2020.json','w')
f.write(dump)
f.close()

### Read .json (avoid running duplicates)

In [136]:
with open('unique_comments2020.json') as f:
    unique_comments2020 = json.load(f)

## Split up longer comments

In [137]:
long_comments = {key:value for key, value in unique_comments2020.items() if len(value) > 5000}

In [139]:
expanded2020 = unique_comments2020.copy()
for key, value in long_comments.items():
    paragraphs = value.split('\n')
    for i in range(len(paragraphs) - 1):
        while i < (len(paragraphs) - 1) and len(paragraphs[i]) < 4800:
            paragraphs[i] += paragraphs[i+1]
            del(paragraphs[i+1])
    for i in range(len(paragraphs)):
        expanded2020[key+'_'+str(i)] = paragraphs[i]
expanded2020 = {key:value for key, value in expanded2020.items() if key not in list(long_comments.keys())}

In [141]:
#{key:len(value) for key, value in expanded2020.items()}

## Remove \r, \n, \s, and any weird or non-ascii characters

In [142]:
expanded2020 = {key:value.replace("\r", " ").replace("\n", " ").replace("\s", " ") for key, value in expanded2020.items()}

In [143]:
for key, value in expanded2020.items():
    expanded2020[key] = ''.join(c for c in value if c in string.printable)

### Write to .json

In [144]:
dump = json.dumps(expanded2020)
f = open('expanded2020.json','w')
f.write(dump)
f.close()

### Read .json (avoid expanding/cleaning comments)

In [145]:
with open('expanded2020.json') as f:
    expanded2020 = json.load(f)